# BDMM HW3 Queries - Group 5

#### Made by:
* Catarina Candeias (m20200656@novaims.unl.pt)
* Catarina Urbano (m20200607@novaims.unl.pt)
* Margarida Pereira (m20201038@novaims.unl.pt) 
* Rita Ferreira (m20200661@novaims.unl.pt)
* Tiago Gonçalves (m20201053@novaims.unl.pt)


In [1]:
from pymongo import MongoClient
import sys, urllib.request, json, pymongo
from bson import json_util

In [2]:
host="rhea.isegi.unl.pt"
port="27022"
user="GROUP_05"
password="if8G1FFeOQICvBc4wz4ZYgft2wlxSnBR"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")
db = client['WWI']

# 1. Web Team

#### - From which state province are our suppliers from?  

In [47]:
query_1 = {
        "$lookup":{
           "from": "Collection_Q1_Embedded_City_StateProvince_Country", # using the previously created embedding
           "localField": "DeliveryCityID", 
           "foreignField": "CityID",
           "as": "DeliveryCity"
        }
    }

query_2 = {
    
    "$group":{"_id":"$DeliveryCity.StateProvince.StateProvinceName",
              
              "count":{                
                  "$sum":1 #grouping by StateProvinceName and counting number of suppliers
              }
              
    }
    
}

query_3 = { "$sort":{"count":-1}}

pipeline=[query_1,query_2, query_3]

In [48]:
list(db.Purchasing_Suppliers.aggregate(pipeline))

[{'_id': [['California']], 'count': 3},
 {'_id': [['Tennessee']], 'count': 2},
 {'_id': [['Missouri']], 'count': 1},
 {'_id': [['South Dakota']], 'count': 1},
 {'_id': [['New Jersey']], 'count': 1},
 {'_id': [['Indiana']], 'count': 1},
 {'_id': [['Kentucky']], 'count': 1},
 {'_id': [['Washington']], 'count': 1},
 {'_id': [['North Carolina']], 'count': 1},
 {'_id': [['Minnesota']], 'count': 1}]

#### - From which state province are the customers who have a higher credit limit?  

In [123]:
query_1 = {
        "$lookup":{
           "from": "Collection_Q1_Embedded_City_StateProvince_Country", # Same as before, joining with the previously created embedding
           "localField": "DeliveryCityID",
           "foreignField": "CityID",
           "as": "DeliveryCity"
        }
    }

query_2 = {
    
    "$group":{"_id":"$DeliveryCity.StateProvince.StateProvinceName", #grouping by stateprovincename
              
              "AverageCreditLimit":{                
                  "$avg":"$CreditLimit" # averaging the credit limit for each stateprovincename
              }
              
    }
    
}

query_3 = { "$sort":{"AverageCreditLimit":-1}} # sorting to get the states with highest credit limit averages

limit = {'$limit':1}

pipeline=[query_1,query_2, query_3, limit]

In [124]:
list(db.Sales_Customers.aggregate(pipeline))

[{'_id': [['Nevada']], 'AverageCreditLimit': 3600.0}]

# 2. Warehouse Group

#### - To know which items get ordered together the most?   

In [41]:
query_1 = {
        "$lookup":{
           "from": "Sales_OrderLines", #joining the collection with itself
           "localField": "OrderID",
           "foreignField": "OrderID",
           "as": "OrderLine2"
        }
    }

query_2={"$unwind":"$OrderLine2"} # unwinding

query_3 = {
    "$addFields": {
        "OrderLine2Description": "$OrderLine2.Description", # getting the orderline2Description and #OrderLine2ID
        "OrderLine2ID": "$OrderLine2.OrderLineID"           # on the root of the document
    }
    

}



query_4 = { "$addFields":{
        "2DescriptionsEqual":{ "$strcasecmp": [ "$Description", "$OrderLine2Description" ] } # checking if the descriptions are equal
    }
}

query_5 = {"$match":{
    
    "2DescriptionsEqual":{"$ne":0} #filtering out collections with OrderLine1_Item = OrderLine2_Item
                                   # so they don't get counted
      
}}



query_6 = {
    
    "$group":{"_id":{"Item1":"$Description","Item2":"$OrderLine2Description"}, # Grouping by pairs of items
              
              "OrdersTogether":{                
                  "$sum":1 # counting the number of orders where they are together
              }
              
    }
    
}

query_7 = {"$sort":{"OrdersTogether":-1}} # sorting in descending order

limit = {"$limit":10} # top 10

pipeline=[query_1, query_2, query_3, query_4, query_5, query_6,  query_7, limit]

In [42]:
list(db.Sales_OrderLines.aggregate(pipeline))

[{'_id': {'Item1': 'DBA joke mug - I will get you in order (White)',
   'Item2': 'Animal with big feet slippers (Brown) M'},
  'OrdersTogether': 15},
 {'_id': {'Item1': 'Animal with big feet slippers (Brown) M',
   'Item2': 'DBA joke mug - I will get you in order (White)'},
  'OrdersTogether': 15},
 {'_id': {'Item1': 'DBA joke mug - two types of DBAs (Black)',
   'Item2': 'Black and orange handle with care despatch tape  48mmx75m'},
  'OrdersTogether': 14},
 {'_id': {'Item1': 'Black and orange handle with care despatch tape  48mmx75m',
   'Item2': 'DBA joke mug - two types of DBAs (Black)'},
  'OrdersTogether': 14},
 {'_id': {'Item1': 'Ride on toy sedan car (Pink) 1/12 scale',
   'Item2': 'Clear packaging tape 48mmx75m'},
  'OrdersTogether': 13},
 {'_id': {'Item1': '"The Gu" red shirt XML tag t-shirt (Black) S',
   'Item2': 'Plush shark slippers (Gray) S'},
  'OrdersTogether': 13},
 {'_id': {'Item1': 'Packing knife with metal insert blade (Yellow) 9mm',
   'Item2': 'Small 9mm replaceme

#### - Which items get ordered the most in bulk (bigger amounts)?  


In [38]:
query_1={"$unwind":"$StockItem"} # unwinding in order to be able to use addfields next

query_2 = {
    "$addFields": {
        "StockItemName": "$StockItem.StockItemName" # moving StockItemName to the root of the document
    }
}    
    
query_3 = {
    
    "$group":{"_id":"$StockItemName", # grouping by StockItemName
              
              "NumberOrders":{                
                  "$sum":1 # counting the number of orders
              },
              "AverageQuantity":{
                  "$avg":"$Quantity"  #getting the average quantity             
              }            
    }   
}

query_4 = {"$sort":{"NumberOrders":-1, "AverageQuantity":-1}} # sorting first by nr_orders (in bulk), and then by avg_qt (bigger amounts)
    
limit = {"$limit":5} # top 5
    
pipeline = [query_1, query_2, query_3, query_4,limit]

In [39]:
list(db.Collection_Q2_Q3_Embedded_OrderLines_Order_StockItems.aggregate(pipeline))

[{'_id': '"The Gu" red shirt XML tag t-shirt (White) XXL',
  'NumberOrders': 394,
  'AverageQuantity': 67.73604060913705},
 {'_id': 'Developer joke mug - this code was generated by a tool (White)',
  'NumberOrders': 392,
  'AverageQuantity': 5.540816326530612},
 {'_id': '"The Gu" red shirt XML tag t-shirt (White) S',
  'NumberOrders': 387,
  'AverageQuantity': 69.64341085271317},
 {'_id': 'Large  replacement blades 18mm',
  'NumberOrders': 386,
  'AverageQuantity': 56.63212435233161},
 {'_id': 'IT joke mug - hardware: part of the computer that can be kicked (Black)',
  'NumberOrders': 386,
  'AverageQuantity': 5.533678756476684}]

#### Which customers have delivery addresses under 10km of distance to the customer with customerID 961?

In [106]:
from pymongo import GEO2D

In [111]:
# getting the coordinates of client961
coords_client961 = list(db.Sales_Customers.find({'CustomerID':961},{'coords':1}))[0]['coords']

In [121]:
# query to get the clients that are within a radius of 10km of client 961
query = {"coords": {"$within": {"$center": [coords_client961, 10]}}} 

list(db.Sales_Customers.find(query, {'_id':0,'CustomerName':1,'PostalAddressLine2':1,'coords':1}))

[{'CustomerName': 'Tailspin Toys (South Laguna, CA)',
  'PostalAddressLine2': 'Aluriville',
  'coords': [33.5006, -117.743]},
 {'CustomerName': 'Eliza Persson',
  'PostalAddressLine2': 'Kommineniville',
  'coords': [33.1803, -117.109]},
 {'CustomerName': 'Tailspin Toys (El Centro, CA)',
  'PostalAddressLine2': 'Lohmusville',
  'coords': [32.792, -115.563]},
 {'CustomerName': 'Edgar Jogi',
  'PostalAddressLine2': 'Shasthriville',
  'coords': [33.3509, -115.73]},
 {'CustomerName': 'Tailspin Toys (Fieldbrook, CA)',
  'PostalAddressLine2': 'Laville',
  'coords': [40.9657, -124.036]},
 {'CustomerName': 'Wingtip Toys (Orick, CA)',
  'PostalAddressLine2': 'Deville',
  'coords': [41.2868, -124.06]},
 {'CustomerName': 'Lakshmi Benipal',
  'PostalAddressLine2': 'Aguilarville',
  'coords': [34.6391, -120.458]},
 {'CustomerName': 'Wingtip Toys (Port Hueneme, CA)',
  'PostalAddressLine2': 'Contiville',
  'coords': [34.1478, -119.195]},
 {'CustomerName': 'Wingtip Toys (Taft Heights, CA)',
  'PostalA

# 3. CFO

#### - Would like to know the monthly order count?  

In [67]:
#query to get the year and the month of the orders
query_1 = {"$project":{'OrderDateYear':{"$year":"$OrderDate"}, 'OrderDateMonth':{"$month":"$OrderDate"}}}

query_2 = {
    
    "$group":{"_id":{"Year":"$OrderDateYear","Month":"$OrderDateMonth"},
              
              "NumberOrders":{                
                  "$sum":1 # grouping by (year,month) and counting nr of orders
              }
              
    }
    
}

query_3 = {
    "$sort":{"_id":1} # sorting by nr of orders
}

pipeline=[query_1,query_2,query_3]

In [68]:
list(db.Sales_Orders.aggregate(pipeline))

[{'_id': {'Year': 2013, 'Month': 1}, 'NumberOrders': 1674},
 {'_id': {'Year': 2013, 'Month': 2}, 'NumberOrders': 1139},
 {'_id': {'Year': 2013, 'Month': 3}, 'NumberOrders': 1683},
 {'_id': {'Year': 2013, 'Month': 4}, 'NumberOrders': 1696},
 {'_id': {'Year': 2013, 'Month': 5}, 'NumberOrders': 1808},
 {'_id': {'Year': 2013, 'Month': 6}, 'NumberOrders': 1675},
 {'_id': {'Year': 2013, 'Month': 7}, 'NumberOrders': 1886},
 {'_id': {'Year': 2013, 'Month': 8}, 'NumberOrders': 1537},
 {'_id': {'Year': 2013, 'Month': 9}, 'NumberOrders': 1617},
 {'_id': {'Year': 2013, 'Month': 10}, 'NumberOrders': 1618},
 {'_id': {'Year': 2013, 'Month': 11}, 'NumberOrders': 1552},
 {'_id': {'Year': 2013, 'Month': 12}, 'NumberOrders': 1565},
 {'_id': {'Year': 2014, 'Month': 1}, 'NumberOrders': 1791},
 {'_id': {'Year': 2014, 'Month': 2}, 'NumberOrders': 1538},
 {'_id': {'Year': 2014, 'Month': 3}, 'NumberOrders': 1586},
 {'_id': {'Year': 2014, 'Month': 4}, 'NumberOrders': 1739},
 {'_id': {'Year': 2014, 'Month': 5}, 

#### Would like to know the average monthly sales prices?

In [126]:
query_1= {
    "$unwind": "$Order" # unwinding order to ease the addfield process
}


query_2 = {
    "$addFields": {
        "OrderDate": "$Order.OrderDate", # getting the orderdate on the root of the document
    }
}

query_3 = {"$project":{'OrderDateYear':{"$year":"$OrderDate"}, # projecting the necessary fields: Year and Month
                       'OrderDateMonth':{"$month":"$OrderDate"}, 
                       'Total':{'$multiply': [ "$Quantity", "$UnitPrice"] # total of the order
                               }
                      }
          }

query_4 = {
    
    "$group":{"_id":{"Year":"$OrderDateYear","Month":"$OrderDateMonth"}, #grouping by (year,month)
              
              "AverageTotal":{                
                  "$avg":"$Total" # average of total
              }
              
    }
    
}

query_5 = {
    "$sort":{"_id":1} # sorting by year_month
}

pipeline=[query_1,query_2,query_3,query_4, query_5]

In [127]:
list(db.Collection_Q2_Q3_Embedded_OrderLines_Order_StockItems.aggregate(pipeline))

[{'_id': {'Year': 2013, 'Month': 1}, 'AverageTotal': 724.2648835447832},
 {'_id': {'Year': 2013, 'Month': 2}, 'AverageTotal': 757.187922705314},
 {'_id': {'Year': 2013, 'Month': 3}, 'AverageTotal': 735.9580812766748},
 {'_id': {'Year': 2013, 'Month': 4}, 'AverageTotal': 782.0304949190817},
 {'_id': {'Year': 2013, 'Month': 5}, 'AverageTotal': 800.637015265836},
 {'_id': {'Year': 2013, 'Month': 6}, 'AverageTotal': 777.4632071937056},
 {'_id': {'Year': 2013, 'Month': 7}, 'AverageTotal': 763.694343622795},
 {'_id': {'Year': 2013, 'Month': 8}, 'AverageTotal': 747.2962440340319},
 {'_id': {'Year': 2013, 'Month': 9}, 'AverageTotal': 767.9943616395371},
 {'_id': {'Year': 2013, 'Month': 10}, 'AverageTotal': 750.3137594275769},
 {'_id': {'Year': 2013, 'Month': 11}, 'AverageTotal': 770.5888844058907},
 {'_id': {'Year': 2013, 'Month': 12}, 'AverageTotal': 750.7256141763995},
 {'_id': {'Year': 2014, 'Month': 1}, 'AverageTotal': 739.6302006335796},
 {'_id': {'Year': 2014, 'Month': 2}, 'AverageTotal'

#### Would like to know the yearly expenditures with suppliers (per supplier name)? 

In [144]:
query_1= {
    "$unwind": "$Supplier" # unwinding Supplier
}

query_2 = {
    "$addFields": {
        "SupplierName": "$Supplier.SupplierName", # adding SupplierName to the root
    }
}

query_3 = {"$project":{'TransactionYear':{"$year":"$TransactionDate"}, # Projecting only the necessary fields
                       'SupplierName':1,
                       'TransactionAmount':1
                      }
          }

query_4 = {
    
    "$group":{"_id":{"Year":"$TransactionYear","Supplier":"$SupplierName"}, # grouping by (year,suppliername)
              
              "Total":{                
                  "$sum":"$TransactionAmount" # getting the total amount of the supplier on the year
              }
              
    }
    
}

pipeline=[query_1,query_2,query_3, query_4]

In [145]:
list(db.Collection_Q4_Embedded_SupplierTransaction_Supplier_SupplierCategory.aggregate(pipeline))

[{'_id': {'Year': 2013, 'Supplier': 'Northwind Electric Cars'},
  'Total': -3.637978807091713e-12},
 {'_id': {'Year': 2013, 'Supplier': 'Litware, Inc.'},
  'Total': 119343.31999999998},
 {'_id': {'Year': 2015, 'Supplier': 'Litware, Inc.'},
  'Total': 1353734.6900000004},
 {'_id': {'Year': 2016, 'Supplier': 'Litware, Inc.'},
  'Total': -1310231.1100000003},
 {'_id': {'Year': 2014, 'Supplier': 'Fabrikam, Inc.'},
  'Total': 1510955.0999999973},
 {'_id': {'Year': 2016, 'Supplier': 'A Datum Corporation'}, 'Total': 0.0},
 {'_id': {'Year': 2016, 'Supplier': 'Fabrikam, Inc.'},
  'Total': -2961969.900000002},
 {'_id': {'Year': 2013, 'Supplier': 'Fabrikam, Inc.'},
  'Total': 485380.49999999977},
 {'_id': {'Year': 2013, 'Supplier': 'Graphic Design Institute'}, 'Total': 0.0},
 {'_id': {'Year': 2014, 'Supplier': 'Litware, Inc.'},
  'Total': 686937.2100000002},
 {'_id': {'Year': 2015, 'Supplier': 'Fabrikam, Inc.'},
  'Total': 2837528.3999999994},
 {'_id': {'Year': 2013, 'Supplier': 'The Phone Compan

# 4. Partnerships

#### - Would like to know what's the most common payment type? 

In [77]:
query_1 =  {
        '$lookup': {
           "from": "Application_PaymentMethods", # joining payment methods to the suppliers
           "localField": "PaymentMethodID",
           "foreignField": "PaymentMethodID",
            "as": "Application_PaymentMethods"
        }
    }

query_2 = {
        '$unwind': '$Application_PaymentMethods' # unwinding to get the methodname on the root of the documents
    }

query_3 = {
    "$addFields": {
        "MethodName": "$Application_PaymentMethods.PaymentMethodName", # getting the methodname to the root
    }
}

query_4 = {
        '$project': {
           'MethodName': 1, # projecting only method name and its id
            'PaymentMethodID':1,
            '_id':0
        }}

query_5 = {
    
    "$group":{"_id":"$MethodName", # grouping by method name
              
              "NrPayments":{                
                  "$sum":1 # getting the number of payments made with a specific method
              }
              
    }
    
}

pipeline=[query_1,query_2,query_3, query_4,query_5]

In [78]:
top_method_suppliers = list(db.Purchasing_SupplierTransactions.aggregate(pipeline))
# getting the top method for supplier transactions

In [79]:
query_1 =  {
        '$lookup': {
           "from": "Application_PaymentMethods", # doing exactly the same, but for the customer transactions
           "localField": "PaymentMethodID",
           "foreignField": "PaymentMethodID",
            "as": "Application_PaymentMethods"
        }
    }

query_2 = {
        '$unwind': '$Application_PaymentMethods'
    }

query_3 = {
    "$addFields": {
        "MethodName": "$Application_PaymentMethods.PaymentMethodName",
    }
}

query_4 = {
        '$project': {
           'MethodName': 1,
            'PaymentMethodID':1,
            '_id':0
        }}

query_5 = {
    
    "$group":{"_id":"$MethodName",
              
              "NrPayments":{                
                  "$sum":1
              }
              
    }
    
}
pipeline=[query_1,query_2,query_3, query_4,query_5]

In [80]:
top_method_customers=list(db.Sales_CustomerTransactions.aggregate(pipeline)) 
# getting the top method for customer transactions

In [89]:
# Most used method and nr of payments made
print(top_method_suppliers[0]['_id'],':',top_method_suppliers[0]['NrPayments'] + top_method_customers[0]['NrPayments'])

EFT : 16447


#### - Which supplier of `Novelty Goods Supplier` has the most transactions? 

In [103]:
query_1= {
    "$unwind": "$Supplier" # unwinding supplier to get its category on the root
}

query_2={"$unwind": "$Supplier.SupplierCategory"} # unwinding suppliercategory to extract the category to the root

query_3 = {
    "$addFields": {
        "SupplierCategoryName": "$Supplier.SupplierCategory.SupplierCategoryName", # cattegoryname to the root
    }
}

query_4 = {"$match":{"SupplierCategoryName":{"$eq":"Novelty Goods Supplier"}}} # matching suppliers of the mentioned category

query_5 = {
    
    "$group":{"_id":"$Supplier.SupplierName",
              
              "NrTransactions":{                
                  "$sum":1 # grouping by supplier name and counting the nr of transactions for that supplier
              }
              
    }
    
}

query_6={"$sort":{"NrTransactions":-1}} # sorting

limit={"$limit":1} # getting our top supplier

pipeline=[query_1,query_2,query_3,query_4,query_5,query_6,limit]

In [104]:
list(db.Collection_Q4_Embedded_SupplierTransaction_Supplier_SupplierCategory.aggregate(pipeline))

[{'_id': 'Graphic Design Institute', 'NrTransactions': 16}]

# 5. Marketing Team

#### - Want to make an appreciation post and needs the name of the sales person with the most invoices in 2013 (person who's customers brought the most money)?

In [58]:
query_1={"$unwind":"$Invoice"} # unwinding invoices

query_2 = {
    "$addFields": {
        "Year": {"$year":"$Invoice.InvoiceDate"}, # getting the year of the invoice to the root of the document
    }
}


query_3 = { "$project":{
    "FullName":"$Invoice.ApplicationPerson.FullName", # getting only the necessary fields: SalespersonName, Year, Revenue
    "Year":1,
    "Revenue":1
     
                       }
       
}

query_4 = {"$match":{"Year":{"$eq":2013}}} # only year 2013

query_5 = {
    
    "$group":{"_id":"$FullName", # grouping by fullname
              
              "NumberInvoices":{                
                  "$sum":1 # counting nr of invoices issued
              },
              "TotalRevenue":{
                  "$sum":"$Revenue"  # summing totalrevenue                
              }            
    }   
}


# sorting by totalrevenue (brought the most money) and number of invoices (with the most invoices)
query_6 = {"$sort":{"TotalRevenue":-1, "NumberInvoices":-1}}

limit={"$limit":10}

pipeline=[query_1,query_2,query_3, query_4, query_5, query_6, limit]

In [59]:
list(db.Collection_Q5_Embedded_InvoiceLines_Invoice_People.aggregate(pipeline))

[{'_id': ['Hudson Onslow'],
  'NumberInvoices': 3148,
  'TotalRevenue': 2545902.55},
 {'_id': ['Archer Lamble'], 'NumberInvoices': 3043, 'TotalRevenue': 2349522.6},
 {'_id': ['Sophia Hinton'], 'NumberInvoices': 3092, 'TotalRevenue': 2316587.9},
 {'_id': ['Anthony Grosse'],
  'NumberInvoices': 3055,
  'TotalRevenue': 2286768.45},
 {'_id': ['Lily Code'], 'NumberInvoices': 2927, 'TotalRevenue': 2280010.2},
 {'_id': ['Jack Potter'], 'NumberInvoices': 2966, 'TotalRevenue': 2244343.75},
 {'_id': ['Amy Trefl'], 'NumberInvoices': 2935, 'TotalRevenue': 2203145.35},
 {'_id': ['Kayla Woodcock'],
  'NumberInvoices': 2917,
  'TotalRevenue': 2187923.45},
 {'_id': ['Hudson Hollinworth'],
  'NumberInvoices': 2975,
  'TotalRevenue': 2124484.15},
 {'_id': ['Taj Shand'], 'NumberInvoices': 2942, 'TotalRevenue': 2113387.3}]